# RAG for Question Similarity in RFPs

## Notebook setup

In [1]:
import pandas as pd

In [2]:
%pip install -qU langchain langchain-openai langchain-cohere


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU qdrant-client lark


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

import dotenv

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [5]:
import textwrap
from IPython.display import HTML, display
from tabulate import tabulate


def _format_cell_text(text, width=50):
    """Private function to format a cell's text."""
    return "\n".join([textwrap.fill(line, width=width) for line in text.split("\n")])


def _format_dataframe_for_tabulate(df):
    """Private function to format the entire DataFrame for tabulation."""
    df_out = df.copy()

    # Format all string columns
    for column in df_out.columns:
        # Check if column is of type object (likely strings)
        if df_out[column].dtype == object:
            df_out[column] = df_out[column].apply(_format_cell_text)
    return df_out


def _dataframe_to_html_table(df):
    """Private function to convert a DataFrame to an HTML table."""
    headers = df.columns.tolist()
    table_data = df.values.tolist()
    return tabulate(table_data, headers=headers, tablefmt="html")


def display_nice(df, num_rows=None):
    """Primary function to format and display a DataFrame."""
    if num_rows is not None:
        df = df.head(num_rows)
    formatted_df = _format_dataframe_for_tabulate(df)
    html_table = _dataframe_to_html_table(formatted_df)
    display(HTML(html_table))

In [53]:
def print_dict_keys(data, indent=0):
    for key, value in data.items():
        print(' ' * indent + str(key))
        if isinstance(value, dict):  # if the value is another dictionary, recurse
            print_dict_keys(value, indent + 4)

## Data preparation

### Load existing RFPs

In [6]:
# List of CSV file paths
existing_rfp_paths = [
    "datasets/rag/rfp_existing_questions_client_2.csv",
]

existing_rfp_df = [pd.read_csv(file_path) for file_path in existing_rfp_paths]

# Concatenate all DataFrames into one
existing_rfp_df = pd.concat(existing_rfp_df, ignore_index=True)

In [7]:
existing_rfp_df

,Project_Title,RFP_Question_ID,RFP_Question,RFP_Answer,Area,Last_Accessed_At,Requester,Status
0,AI-Powered Risk Assessment Model Development f...,1,Can you discuss your expertise in creating AI-...,Our company has 15 years of experience in deve...,General,18/12/2022,Bank B,Awarded
1,AI-Powered Risk Assessment Model Development f...,2,How do you keep your AI applications current w...,We maintain a dedicated R&D team focused on in...,General,18/12/2022,Bank B,Awarded
2,AI-Powered Risk Assessment Model Development f...,3,Are your AI applications adaptable to specific...,"Absolutely, customization is a core aspect of ...",General,18/12/2022,Bank B,Awarded
3,AI-Powered Risk Assessment Model Development f...,4,What steps do you undertake to protect user pr...,User privacy and data security are paramount. ...,General,18/12/2022,Bank B,Awarded
4,AI-Powered Risk Assessment Model Development f...,5,What strategies do you employ to design user i...,Our design philosophy centers on simplicity an...,General,18/12/2022,Bank B,Awarded
5,AI-Powered Risk Assessment Model Development f...,6,Explain the support and maintenance services y...,"Post-launch, we offer comprehensive support an...",General,18/12/2022,Bank B,Awarded
6,AI-Powered Risk Assessment Model Development f...,7,How do you evaluate the effectiveness and impa...,Success measurement is tailored to each projec...,General,18/12/2022,Bank B,Awarded
7,AI-Powered Risk Assessment Model Development f...,8,How do you manage ethical concerns in your LLM...,We adhere to ethical AI practices by implement...,Large Language Models,18/12/2022,Bank B,Awarded
8,AI-Powered Risk Assessment Model Development f...,9,"Could you outline how you train your LLMs, inc...",Our LLM training process begins with the metic...,Large Language Models,18/12/2022,Bank B,Awarded
9,AI-Powered Risk Assessment Model Development f...,10,How do you ensure your LLMs continuously learn...,We implement advanced continuous learning mech...,Large Language Models,18/12/2022,Bank B,Awarded


In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader

documents = []

# Iterate through each file path in the list
for file_path in existing_rfp_paths:
    loader = CSVLoader(
        file_path=file_path,
        metadata_columns=["Area"]
    )

    # Load a document from the current CSV file
    doc = loader.load()
    
    # Append documents
    documents.extend(doc)

When using `CSVLoader`, each document represents a single row and includes its respective contents:

In [9]:
number_of_documents = 5

for i, document in enumerate(documents[:number_of_documents]):
    print(f"Document {i + 1}: {document}")

Document 1: page_content='Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing\nRFP_Question_ID: 1\nRFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?\nRFP_Answer: Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For instance, our project MediAI Insight for the healthcare industry demonstrated significant achievements in patient data analysis, resulting in a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. Our platform has engaged over 200 healthcare facilities, achieving a user satisfaction rate of 95%.\nLast_Accessed_At: 18/12/2022\nRequester: Bank B\nStatus: Awarded' metadata={'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General'}
Document 2: page_content='Project_Title: AI-Powered Risk Assessment

Accessing the page content of each document:

In [10]:
number_of_documents = 2

for i, document in enumerate(documents[:number_of_documents]):
    print(f"Page content for document {i + 1}:")
    print(document.page_content)
    print()

Page content for document 1:
Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing
RFP_Question_ID: 1
RFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?
RFP_Answer: Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For instance, our project MediAI Insight for the healthcare industry demonstrated significant achievements in patient data analysis, resulting in a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. Our platform has engaged over 200 healthcare facilities, achieving a user satisfaction rate of 95%.
Last_Accessed_At: 18/12/2022
Requester: Bank B
Status: Awarded

Page content for document 2:
Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing
RFP_Question_ID: 2
RFP_Question: How do you keep your AI appli

Note that when adding metadata, it is appended to the default metadata, which consists of the row number and the source: 

In [11]:
number_of_documents = 5

for i, document in enumerate(documents[:number_of_documents]):
    print(f"Metadata for document {i + 1}: {document.metadata}")

Metadata for document 1: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General'}
Metadata for document 2: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 1, 'Area': 'General'}
Metadata for document 3: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 2, 'Area': 'General'}
Metadata for document 4: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 3, 'Area': 'General'}
Metadata for document 5: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 4, 'Area': 'General'}


## Split the documents into chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, add_start_index=True
)
chunks = text_splitter.split_documents(documents)

Get some general information about the chunks:

In [13]:
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 98


See the length of the bigger and smaller chunks:

In [14]:
max_chunk_length = max([len(chunk.page_content) for chunk in chunks])
min_chunk_length = min([len(chunk.page_content) for chunk in chunks])
mean_chunk_length = sum([len(chunk.page_content) for chunk in chunks]) / len(chunks)

print(f"Maximum chunk length: {max_chunk_length}")
print(f"Minimum chunk length: {min_chunk_length}")
print(f"Mean chunk length: {mean_chunk_length}")

Maximum chunk length: 499
Minimum chunk length: 12
Mean chunk length: 267.6020408163265


Plot the distribution of chunks: 

In [15]:
import plotly.express as px

# Calculate lengths of each chunk's page_content
chunk_lengths = [len(chunk.page_content) for chunk in chunks]

# Creating a histogram of chunk lengths
fig = px.histogram(chunk_lengths, nbins=50, title="Distribution of Chunk Lengths")
fig.update_layout(
    xaxis_title="Chunk Length",
    yaxis_title="Count",
    bargap=0.2,
    showlegend=False
)

# Add summary statistics as text on the plot
fig.add_annotation(
    x=max(chunk_lengths),
    y=0,
    showarrow=False,
    yshift=10
)

# Show the plot
fig.show()

Inspect the chunks: 

In [16]:
number_of_chunks = 5  

for index, chunk in enumerate(chunks[:i]):
    print(f"Chunk {index + 1}: {chunk}")  

Chunk 1: page_content='Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing\nRFP_Question_ID: 1\nRFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?' metadata={'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General', 'start_index': 0}
Chunk 2: page_content='RFP_Answer: Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For instance, our project MediAI Insight for the healthcare industry demonstrated significant achievements in patient data analysis, resulting in a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. Our platform has engaged over 200 healthcare facilities, achieving a user satisfaction rate of' metadata={'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General',

See the page content of each chunk:

In [17]:
number_of_chunks = 5

for i, document in enumerate(chunks[:number_of_chunks]):
    print(f"Page content for chunk {i + 1}:")
    print(document.page_content)
    print()

Page content for chunk 1:
Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing
RFP_Question_ID: 1
RFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?

Page content for chunk 2:
RFP_Answer: Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For instance, our project MediAI Insight for the healthcare industry demonstrated significant achievements in patient data analysis, resulting in a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. Our platform has engaged over 200 healthcare facilities, achieving a user satisfaction rate of

Page content for chunk 3:
rate of 95%.

Page content for chunk 4:
Last_Accessed_At: 18/12/2022
Requester: Bank B
Status: Awarded

Page content for chunk 5:
Project_Title: AI-Powered Risk Assessment Model Development

See the metadata for individual chunks:

In [18]:
number_of_chunks = 5  

for i, chunk in enumerate(chunks[:number_of_chunks]):
    print(f"Metadata for chunk {i + 1}: {chunk.metadata}")



Metadata for chunk 1: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General', 'start_index': 0}
Metadata for chunk 2: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General', 'start_index': 234}
Metadata for chunk 3: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General', 'start_index': 723}
Metadata for chunk 4: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 0, 'Area': 'General', 'start_index': 736}
Metadata for chunk 5: {'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'row': 1, 'Area': 'General', 'start_index': 0}


Access the source of each chunk:

In [19]:
number_of_chunks = 5  

for i, chunk in enumerate(chunks[:number_of_chunks]):
    print(f"Source for chunk {i + 1}: {chunk.metadata['source']}")

Source for chunk 1: datasets/rag/rfp_existing_questions_client_2.csv
Source for chunk 2: datasets/rag/rfp_existing_questions_client_2.csv
Source for chunk 3: datasets/rag/rfp_existing_questions_client_2.csv
Source for chunk 4: datasets/rag/rfp_existing_questions_client_2.csv
Source for chunk 5: datasets/rag/rfp_existing_questions_client_2.csv


## Store chunks into a vectorstore

In [20]:
from langchain.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
)

## Create evaluation dataset

In [21]:
# Load all RFPs into a single pandas DataFrame

rag_evaluation_df = pd.read_csv("datasets/rag/rag_evaluation_dataset_v1.csv")

display_nice(rag_evaluation_df, num_rows=2)


id,new_rfp,new_question,question_to_llm,answer,ground_truth,existing_rfp,llm
1,rfp_new_questions_client_100.csv,"What is your experience in developing AI-based applications, and can you provide examples of successful projects?","What is the most similar question to: ""What is your experience in developing AI-based applications, and can you provide examples of successful projects?""",None,Can you discuss your expertise in creating AI- driven applications and share examples of your successful implementations?,rfp_exisiting_questions_client_2.csv,gpt-4-turbo
2,rfp_new_questions_client_100.csv,How do you ensure your AI-based apps remain up-to- date with the latest AI advancements and technologies?,"What is the most similar question to: ""How do you ensure your AI-based apps remain up-to-date with the latest AI advancements and technologies?""",None,How do you keep your AI applications current with ongoing advancements in artificial intelligence?,rfp_exisiting_questions_client_2.csv,gpt-4-turbo


In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.0)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

In [23]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. 
If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [28]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

# Step 1: "question": Retrieved from the "question" key.
# Step 2: "context": Retrieved from the "question" key and fed into the retriever.
# Step 3: "context": Assigned to a RunnablePassthrough object using the "context" key from the previous step.
# Step 4: "answer": "context" and "question" are combined to format the prompt, then sent to the LLM and stored under the "answer" key.
# Step 5: "context": Repopulated using the "context" key from the previous step.

llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

rag_chain = (
    
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"answer": prompt | llm, "context": itemgetter("context")}

)

Ask a question to test the chain:

In [30]:
question = "Find a similar question as this one: 'What is your experience in developing AI-based applications?'"
response = rag_chain.invoke({"question" : question})
print(response)

{'answer': AIMessage(content='RFP_Question_ID: 1\nRFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 2076, 'total_tokens': 2108}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_76f018034d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0fb67830-d0c7-4248-86d8-18a55f495ae4-0'), 'context': [Document(page_content='Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing\nRFP_Question_ID: 1\nRFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?', metadata={'Area': 'General', 'row': 0, 'source': 'datasets/rag/rfp_existing_questions_client_2.csv', 'start_index': 0}), Document(page_content='Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing\nRFP_Question_ID: 2\nRFP_Question: How do you ke

As defined in the earlier chat prompt, the RAG response includes two fields: `answer` and `context`:

In [52]:
print_dict_keys(response)

answer
context


Inspecting the answer, we see that the `rag_chain` is functioning correctly and identifies the most similar question in the `vectorstore`:

In [67]:
print(f"Question:")
print(question)
print()
print(f"Answer:")
print(response["answer"].content)

Question:
Find a similar question as this one: 'What is your experience in developing AI-based applications?'

Answer:
RFP_Question_ID: 1
RFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?


Next, we inspect the content of the `answer` and the `context` retrieved based on the `question`. The context should contain `k` chunks, the most relevant based on the question. Remember that we set`k` in the `retriever` earlier. These `k` chunks are pasted into the prompt as text, informing the LLM to generate an answer that is closer in the embedding space to the question.

In [63]:
number_of_chunks = 5  

for i, chunk in enumerate(response["context"][:number_of_chunks]):
    print(f"Content for chunk {i + 1}:")  # i + 1 to start counting from 1 instead of 0
    print(chunk.page_content)
    print()

Content for chunk 1:
Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing
RFP_Question_ID: 1
RFP_Question: Can you discuss your expertise in creating AI-driven applications and share examples of your successful implementations?

Content for chunk 2:
Project_Title: AI-Powered Risk Assessment Model Development for Loan Processing
RFP_Question_ID: 2
RFP_Question: How do you keep your AI applications current with ongoing advancements in artificial intelligence?

Content for chunk 3:
RFP_Answer: Our company has 15 years of experience in developing AI-based applications, with a strong portfolio in sectors such as healthcare, finance, and education. For instance, our project MediAI Insight for the healthcare industry demonstrated significant achievements in patient data analysis, resulting in a 30% reduction in diagnostic errors and a 40% improvement in treatment personalization. Our platform has engaged over 200 healthcare facilities, achieving a user satisfaction 

We now inspect the `response_metadata` object to understand its contents and identify what could be useful to incorporate in our RAG evaluation dataset:

In [61]:
print(response["answer"].response_metadata)

{'token_usage': {'completion_tokens': 32, 'prompt_tokens': 2076, 'total_tokens': 2108}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_76f018034d', 'finish_reason': 'stop', 'logprobs': None}


In [60]:
print_dict_keys(response["answer"].response_metadata)

token_usage
    completion_tokens
    prompt_tokens
    total_tokens
model_name
system_fingerprint
finish_reason
logprobs


In [66]:
print(response["source_documents"])

KeyError: 'source_documents'

In [ ]:
#if 'source_documents' not in rag_evaluation_df.columns:
#    rag_evaluation_df['source_documents'] = pd.Series([[] for _ in range(len(rag_evaluation_df))], index=rag_evaluation_df.index)


# Iterate over the DataFrame rows
for i, row in rag_evaluation_df.iterrows():
    print(f"Processing row {i}...")

    # Check if the 'answer' field is 'None' (as a string) for the current row
    if row["answer"] == "None":
        print(f"Answer is 'None' for question ID {i}. Invoking RAG model...")

        # Invoke the RAG model with the question from the current row
        response = rag_chain.invoke(row["question_to_llm"])

        # Store whatever response comes from the LLM
        rag_evaluation_df.at[i, "answer"] = response["answer"]
        print(f"Question ID {i} answer updated with the response from the RAG model.")

        # Compute the hashed metadata for each source document
        source_hashes = [stable_hash_meta(source_document.metadata) for source_document in response["source_documents"]]

        # Check if source_hashes is a list before assignment, throw error if not
        #if not isinstance(source_hashes, list):
        #    print(f"Expected a list for source_hashes but got {type(source_hashes)} instead.")
        #    raise TypeError(f"source_hashes must be a list, but was {type(source_hashes)}")

        # Assign the list of source document hashes directly to the DataFrame cell
        #rag_evaluation_df.at[i, "source_documents"] = source_hashes
        #print(f"Question ID {i} source documents updated with hashed metadata.")

print("Processing complete.")


In [ ]:
display_nice(rag_evaluation_df, num_rows=5)

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

In [ ]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

In [ ]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [ ]:
run = False
if run:

    from typing import List

    from langchain_core.runnables import RunnableParallel, RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser

    # This function formats a list of Document objects into a single string.
    # Each document's content and source are formatted and separated by two newlines.
    def format_docs(docs: List[Document]) -> str:
        return "\n\n".join(
            f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs
        )

    # This chain is used for processing 'source_documents'. It formats the documents
    # using the 'format_docs' function, then passes the formatted string through
    # subsequent unspecified operations (`prompt`, `llm`) and finally parses the output to a string.
    rag_chain_from_docs = (
        RunnablePassthrough.assign(
            source_documents=(lambda x: format_docs(x["source_documents"]))
        )
        | prompt  # Uses the 'prompt' template to format the context and question.
        | llm     # Uses the language model to generate an answer.
        | StrOutputParser()  # Parses the output from the language model into a string format.
    )

    # This RunnableParallel constructs a parallel chain for processing.
    # It takes 'source_documents' from a retriever and a 'question' as inputs.
    # The 'answer' part of the chain is assigned to the previously defined 'rag_chain_from_docs'.
    rag_chain = RunnableParallel(
        {
            "source_documents": retriever,  
            "question": RunnablePassthrough(),  # Passes the question through without modification.
        }
    ).assign(answer=rag_chain_from_docs)  # The final output is determined by the chain that processes documents.

In [ ]:
import hashlib
import json
from langchain_core.documents import Document

def stable_hash_meta(doc: Document) -> str:
    """
    Stable hash document based on its metadata. Assumes 'metadata' is always present.
    """
    try:
        metadata_json = json.dumps(doc.metadata, sort_keys=True)
    except AttributeError:
        raise ValueError("Document does not have metadata.")
    return hashlib.sha1(metadata_json.encode()).hexdigest()

In [ ]:
#splits_ids = [{"doc": split, "id": stable_hash_meta(split)} for split in splits]

#existing_ids = vectorstore.get()["ids"]

#new_splits_ids = [split for split in splits_ids if split["id"] not in existing_ids]